In [2]:
%config IPCompleter.greedy=True

In [55]:
import os
import pandas as pd

data_dir = "../data/"
datasets_fn = [
    (data_dir + file)
    for file in os.listdir(data_dir)
]

datasets = [
    pd.read_csv(file_name)
    for file_name in datasets_fn
]

datasets[0]

,transactionid,commitdate,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp,bug
0,3,2001/12/12 17:41,1,1,3,0.579380,0.093620,0.000000,480.666667,1,14,596,0.666667,143,133.500000,129,1
1,7,1999/10/12 12:57,1,1,1,0.000000,0.000000,0.000000,398.000000,1,1,0,1.000000,140,140.000000,137,1
2,8,2002/5/15 16:55,3,3,52,0.739279,0.183477,0.208913,283.519231,0,23,15836,0.750000,984,818.650000,978,0
3,9,2002/1/21 15:37,1,1,8,0.685328,0.016039,0.012880,514.375000,1,21,1281,1.000000,579,479.250000,550,0
4,10,2001/12/19 16:44,2,2,38,0.769776,0.091829,0.072746,366.815789,1,21,6565,0.763158,413,313.250000,405,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,20762,2002/7/26 16:44,1,1,1,0.000000,0.025698,0.039106,895.000000,1,21,0,1.000000,219,190.083333,212,1
4616,20840,2002/11/13 3:29,1,1,2,0.918296,0.005450,0.002725,183.500000,1,18,21,1.000000,355,252.833333,335,1
4617,20936,2005/9/27 12:54,1,1,1,0.000000,0.000000,0.000000,2.000000,1,2,0,1.000000,961,239.861905,3,0
4618,20937,2005/9/27 13:03,1,1,1,0.000000,21.000000,0.000000,2.000000,1,2,0,1.000000,615,464.500000,3,0


In [58]:
from sklearn.model_selection import KFold

kf_splitter = KFold(n_splits=10, shuffle=False, random_state=None)

def split_train_data(dataset):
    """Returns all the experiments with each experiment represented
    as a tuple: (training data, testing data)"""
    return [
        (dataset.iloc[train_indicies], dataset.iloc[test_indicies])
        for train_indicies, test_indicies in kf_splitter.split(dataset)
    ]

# Contains (train, test) for all data sets
splitted_data = [
    split_train_data(data)
    for data in datasets
]

data_train, data_test = splitted_data[0][0]
data_train

,transactionid,commitdate,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp,bug
462,1012,2002/7/6 5:33,1,1,1,0.000000,0.021978,0.000000,182.000000,1,20,3,1.000000,7,7.000000,7,1
463,1014,2002/6/14 12:56,1,1,1,0.000000,0.005525,0.005525,181.000000,1,19,37,1.000000,88,68.000000,86,0
464,1015,2002/5/8 17:25,1,1,1,0.000000,0.005525,0.000000,181.000000,0,19,0,1.000000,180,156.333333,173,0
465,1016,2002/5/8 15:02,1,1,1,0.000000,0.005556,0.011111,180.000000,1,19,14,1.000000,179,155.333333,172,0
466,1017,2002/4/24 3:24,2,2,28,0.934459,0.021663,0.029374,194.535714,1,22,974,0.785714,118,94.333333,118,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,20762,2002/7/26 16:44,1,1,1,0.000000,0.025698,0.039106,895.000000,1,21,0,1.000000,219,190.083333,212,1
4616,20840,2002/11/13 3:29,1,1,2,0.918296,0.005450,0.002725,183.500000,1,18,21,1.000000,355,252.833333,335,1
4617,20936,2005/9/27 12:54,1,1,1,0.000000,0.000000,0.000000,2.000000,1,2,0,1.000000,961,239.861905,3,0
4618,20937,2005/9/27 13:03,1,1,1,0.000000,21.000000,0.000000,2.000000,1,2,0,1.000000,615,464.500000,3,0


In [69]:
def split_input_output(data):
    """Split the data into the information and the answer"""
    X = data
    X = X.drop('bug', axis=1)
    X = X.drop('transactionid', axis=1)
    X = X.drop('commitdate', axis=1)
    Y = data['bug']
    return (X, Y)

input_output_data = [
    [
        (split_input_output(train_data), split_input_output(test_data))
        for train_data, test_data in dataset
    ]
    for dataset in splitted_data
]

# First dataset, First experiment, Training data, data for training
input_output_data[0][0][0][0]

,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp
462,1,1,1,0.000000,0.021978,0.000000,182.000000,1,20,3,1.000000,7,7.000000,7
463,1,1,1,0.000000,0.005525,0.005525,181.000000,1,19,37,1.000000,88,68.000000,86
464,1,1,1,0.000000,0.005525,0.000000,181.000000,0,19,0,1.000000,180,156.333333,173
465,1,1,1,0.000000,0.005556,0.011111,180.000000,1,19,14,1.000000,179,155.333333,172
466,2,2,28,0.934459,0.021663,0.029374,194.535714,1,22,974,0.785714,118,94.333333,118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,1,1,1,0.000000,0.025698,0.039106,895.000000,1,21,0,1.000000,219,190.083333,212
4616,1,1,2,0.918296,0.005450,0.002725,183.500000,1,18,21,1.000000,355,252.833333,335
4617,1,1,1,0.000000,0.000000,0.000000,2.000000,1,2,0,1.000000,961,239.861905,3
4618,1,1,1,0.000000,21.000000,0.000000,2.000000,1,2,0,1.000000,615,464.500000,3


In [ ]:
from sklearn import ensemble

params = {
    'n_estimators': 1200,
    'max_depth': 3,
    'subsample': 0.5,
    'learning_rate': 0.01,
    'min_samples_leaf': 1,
    'random_state': 3
}
clf = ensemble.GradientBoostingClassifier(**params)

# First dataset, First experiment, Training data, data for training
# input_output_data[0][0][0][0]

for ds_num, dataset in enumerate(input_output_data):
    for exp_num, experiment in enumerate(dataset):
        training, testing = experiment
        
        x_train = training[0]
        y_train = training[1]
        
        x_test = testing[0]
        y_test = testing[1]
        
        clf.fit(x_train, y_train)
        acc = clf.score(x_test, y_test)
        print("DS: {}\tExp {}\tAccuracy: {:.4f}".format(ds_num, exp_num, acc))
        
        
# X_train = input_output_data[0][0][0][0]
# y_train = input_output_data[0][0][0][1]

# X_test = input_output_data[0][0][1][0]
# y_test = input_output_data[0][0][1][1]

# clf.fit(X_train, y_train)
# acc = clf.score(X_test, y_test)
# print("Accuracy: {:.4f}".format(acc))

DS: 0	Exp 0	Accuracy: 0.7792
DS: 0	Exp 1	Accuracy: 0.7619
DS: 0	Exp 2	Accuracy: 0.7316
DS: 0	Exp 3	Accuracy: 0.7338
DS: 0	Exp 4	Accuracy: 0.7208
DS: 0	Exp 5	Accuracy: 0.7706
DS: 0	Exp 6	Accuracy: 0.7662
DS: 0	Exp 7	Accuracy: 0.7965
DS: 0	Exp 8	Accuracy: 0.7684
DS: 0	Exp 9	Accuracy: 0.7706
DS: 1	Exp 0	Accuracy: 0.7691
DS: 1	Exp 1	Accuracy: 0.8274
DS: 1	Exp 2	Accuracy: 0.7691
DS: 1	Exp 3	Accuracy: 0.7623
DS: 1	Exp 4	Accuracy: 0.8072
DS: 1	Exp 5	Accuracy: 0.7483
DS: 1	Exp 6	Accuracy: 0.7843
DS: 1	Exp 7	Accuracy: 0.7191
DS: 1	Exp 8	Accuracy: 0.7775
DS: 1	Exp 9	Accuracy: 0.7169
DS: 2	Exp 0	Accuracy: 0.8601
DS: 2	Exp 1	Accuracy: 0.8584
